In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import json
from itertools import product
import time


# Define Modified Network (Batch Norm + Residual Connections)
class ModifiedNet(nn.Module):
    def __init__(self):
        super(ModifiedNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.bn2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 32, 3)
        self.bn3 = nn.BatchNorm2d(32)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 2 * 2, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        residual = x
        x = self.pool(torch.relu(self.bn1(self.conv1(x))))
        residual = self.pool(self.bn1(self.conv1(residual)))
        x = x + residual
        residual = x
        x = self.pool(torch.relu(self.bn2(self.conv2(x))))
        residual = self.pool(self.bn2(self.conv2(residual)))
        x = x + residual
        x = self.pool(torch.relu(self.bn3(self.conv3(x))))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


# Load the CIFAR-10 dataset
def get_dataloaders(batch_size):
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(
        trainset, batch_size=batch_size, shuffle=True, num_workers=2)

    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(
        testset, batch_size=batch_size, shuffle=False, num_workers=2)

    return trainloader, testloader


# Train and Evaluate the Model
def train_and_evaluate(net, trainloader, testloader, optimizer, criterion, epochs, device, classes):
    net.to(device)
    train_loss = []

    for epoch in range(epochs):
        running_loss = 0.0
        net.train()
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            labels_one_hot = torch.nn.functional.one_hot(labels, num_classes=len(classes)).float()

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, labels_one_hot)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        train_loss.append(running_loss / len(trainloader))

    # Evaluate on validation set
    net.eval()
    correct = 0
    total = 0
    class_correct = [0 for _ in range(len(classes))]
    class_total = [0 for _ in range(len(classes))]

    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            for i in range(len(labels)):
                label = labels[i]
                pred = predicted[i]
                if label == pred:
                    class_correct[label] += 1
                class_total[label] += 1

    overall_accuracy = 100 * correct / total
    per_class_accuracy = {classes[i]: 100 * class_correct[i] / class_total[i] if class_total[i] > 0 else 0
                          for i in range(len(classes))}

    return train_loss, overall_accuracy, per_class_accuracy

In [3]:
import optuna
results = []
# Define an objective function to be minimized.
def objective(trial):
    net = ModifiedNet()
    epochs = trial.suggest_categorical('epochs', [20, 25, 30])
    batch_size = trial.suggest_categorical('batch_size', [64, 128])
    lr = trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True)
    optimizer_type = trial.suggest_categorical('optimizer', ['Adam'])
    optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    if optimizer_type == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    elif optimizer_type == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr=lr)
    trainloader, testloader = get_dataloaders(batch_size)
    criterion = nn.CrossEntropyLoss()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    print(f"Running experiment: lr={lr}, batch_size={batch_size}, optimizer={optimizer_type}, "
          f"loss_function={'CrossEntropy'}, architecture={'ModifiedNet'}, epochs={epochs}")
    start_time = time.time()
    train_loss, overall_accuracy, per_class_accuracy = train_and_evaluate(
        net, trainloader, testloader, optimizer, criterion, epochs, device, classes)
    end_time = time.time()

    acc = overall_accuracy
    results.append({
        'learning_rate': lr,
        'batch_size': batch_size,
        'optimizer': optimizer_type,
        'loss_function': 'CrossEntropy',
        'architecture': 'ModifiedNet',
        'epochs': epochs,
        'train_loss': train_loss,
        'overall_accuracy': overall_accuracy,
        'per_class_accuracy': per_class_accuracy,
        'time_taken': end_time - start_time
    })

    return acc  # An objective value linked with the Trial object.

study = optuna.create_study(direction="maximize")  # Create a new study.
study.optimize(objective, n_trials=50)  # Invoke optimization of the objective function.

[I 2024-11-29 21:53:44,906] A new study created in memory with name: no-name-f8a77320-671d-437d-894d-ba07f7e078dd


Files already downloaded and verified
Files already downloaded and verified
Running experiment: lr=0.00011536404620534518, batch_size=128, optimizer=Adam, loss_function=CrossEntropy, architecture=ModifiedNet, epochs=20


[I 2024-11-29 21:57:18,204] Trial 0 finished with value: 64.94 and parameters: {'epochs': 20, 'batch_size': 128, 'learning_rate': 0.00011536404620534518, 'optimizer': 'Adam'}. Best is trial 0 with value: 64.94.


Files already downloaded and verified
Files already downloaded and verified
Running experiment: lr=0.004161511902508829, batch_size=128, optimizer=Adam, loss_function=CrossEntropy, architecture=ModifiedNet, epochs=30


[I 2024-11-29 22:02:17,367] Trial 1 finished with value: 71.34 and parameters: {'epochs': 30, 'batch_size': 128, 'learning_rate': 0.004161511902508829, 'optimizer': 'Adam'}. Best is trial 1 with value: 71.34.


...

[I 2024-11-30 00:59:37,837] Trial 47 finished with value: 70.73 and parameters: {'epochs': 20, 'batch_size': 128, 'learning_rate': 0.0005700923227984256, 'optimizer': 'Adam'}. Best is trial 36 with value: 72.16.


Files already downloaded and verified
Files already downloaded and verified
Running experiment: lr=0.000767706340331489, batch_size=64, optimizer=Adam, loss_function=CrossEntropy, architecture=ModifiedNet, epochs=30


[I 2024-11-30 01:03:56,319] Trial 48 finished with value: 71.37 and parameters: {'epochs': 30, 'batch_size': 64, 'learning_rate': 0.000767706340331489, 'optimizer': 'Adam'}. Best is trial 36 with value: 72.16.


Files already downloaded and verified
Files already downloaded and verified
Running experiment: lr=0.000428339547535551, batch_size=128, optimizer=Adam, loss_function=CrossEntropy, architecture=ModifiedNet, epochs=20


[I 2024-11-30 01:07:22,440] Trial 49 finished with value: 70.06 and parameters: {'epochs': 20, 'batch_size': 128, 'learning_rate': 0.000428339547535551, 'optimizer': 'Adam'}. Best is trial 36 with value: 72.16.


In [4]:
with open('experiment_results_12.json', 'w') as f:
    json.dump(results, f, indent=4)

print("All experiments completed and results saved to 'experiment_results.json'")

All experiments completed and results saved to 'experiment_results.json'
